In [8]:
import math
import time

input_str = """
ADD_DRIVER D1 1 1
ADD_DRIVER D2 4 5
ADD_DRIVER D3 2 2
ADD_RIDER R1 0 0
MATCH R1
START_RIDE RIDE-001 2 R1
STOP_RIDE RIDE-001 4 5 32
BILL RIDE-001
"""

input_str = """

ADD_DRIVER D1 0 1
ADD_DRIVER D2 2 3
ADD_RIDER R1 3 5
ADD_DRIVER D3 4 2
ADD_RIDER R2 1 1
MATCH R1
MATCH R2
START_RIDE RIDE-101 1 R1
START_RIDE RIDE-102 1 R2
STOP_RIDE RIDE-101 10 2 48
STOP_RIDE RIDE-102 7 9 50
BILL RIDE-101
BILL RIDE-102

"""

testcase = input_str.strip().splitlines()#[i for i in  input_str.splitlines() if len(i)>0]

In [9]:
from dataclasses import dataclass,field

from datetime import datetime

from typing import Tuple,Dict,DefaultDict,List

from collections import defaultdict

In [10]:
from icecream import ic

In [11]:
@dataclass(slots=True)
class Ride:
    id: str
    driver_id: str
    rider_id: str

    # driver_coords: Tuple[int]
    start_coords: Tuple[int]
    dest_coords: Tuple[int] = field(init=False)

    started_at: datetime = field(init=False)
    ended_at: datetime = field(init=False)

    duration: int = field(init=False)

    def start(self):
        self.started_at = datetime.now()

    def stop(self,dest_coords,duration_in_minutes=0):
        self.ended_at = datetime.now()
        self.dest_coords = dest_coords
        self.duration = duration_in_minutes

In [12]:
@dataclass(slots=True)
class Driver:
    id: str
    coord: List[int]

@dataclass(slots=True)
class Rider:
    id: str
    coord: List[int]


In [13]:
drivers:Dict[str,Driver] = dict()

matched:DefaultDict[str,List[int]] = defaultdict(lambda:[]) #driverid's

riders:Dict[str,Rider] = dict()

rides:Dict[str,Ride] = dict()

def rest_global():
    # gs = [drivers,matched,riders,rides]
    drivers.clear()
    matched.clear()
    riders.clear()
    rides.clear()

In [14]:
def add_driver(id:str,x:int,y:int):
    driver = Driver(id,[x,y])
    drivers[id] = driver

    ic(driver)

In [15]:
def add_rider(id:str, x:int, y:int):
    rider = Rider(id,[x,y])
    riders[id] = rider

    ic(rider)

In [16]:
def match(rider_id:str):

    drivers_available = []

    rider = riders[rider_id]

    for driver in drivers.values():
        d = distance(driver.coord,rider.coord)
        if d<=5:
            ic(rider_id,driver.id,d)
            drivers_available.append((driver.id,d))
        else:
            ic(f"ignoring {driver.id} is {d} away")


    drivers_available = sorted(drivers_available,key =lambda val : (val[1],val[0]))

    ic(drivers_available)

    if len(drivers_available)==0:
        print("NO_DRIVERS_AVAILABLE")
        return

    drivers_available = drivers_available[:5]

    shorted_drivers = [i[0] for i in drivers_available]

    matched[rider_id] = shorted_drivers

    print(f"""DRIVERS_MATCHED {" ".join(shorted_drivers)}""")


In [17]:
def start_ride(ride_id:str, driver_n:int, rider_id:str):
    if ride_id in rides:
        print("INVALID_RIDE")
        ic(f"ride-{ride_id} already exist")
        return

    drivers_matched = matched[rider_id]

    if len(drivers_matched)<driver_n:
        print("INVALID_RIDE")
        ic(f"no drivers matched")
        return

    driver_id = drivers_matched[driver_n-1]

    rider = riders[rider_id]

    ride = Ride(ride_id,driver_id,rider.id,rider.coord)
    rides[ride_id] = ride
    ride.start()

    print(f"RIDE_STARTED {ride.id}")

In [18]:
def stop_ride(ride_id:str, dest_coords:Tuple[int], time_taken:int):
    if ride_id not in rides:
        print("INVALID_RIDE")
        return
    ride = rides[ride_id]
    ride.dest_coords = dest_coords
    ride.stop(dest_coords,time_taken)
    ic(ride)
    print(f"RIDE_STOPPED {ride_id}")

In [19]:
def distance(start_coord: Tuple[int],dest_coord:Tuple[int])->float:
    x2, x1 = dest_coord[0],start_coord[0]
    y2, y1 = dest_coord[1],start_coord[1]

    return math.sqrt((x2-x1)**2+(y2-y1)**2)

def bill(ride_id:str):
    base_fare = 50
    cost_per_km = 6.5
    cost_per_min = 2
    service_tax = .2

    if ride_id not in rides:
        print("INVALID_RIDE")
        return

    ride:Ride = rides[ride_id]

    if ride.ended_at is None:
        print("RIDE_NOT_COMPLETED")
        return

    # duration = (ride.started_at-ride.ended_at).total_seconds()//60 #FIXME could be a prob

    duration = ride.duration

    d = round(distance(ride.start_coords, ride.dest_coords),2)

    amt= base_fare + duration*cost_per_min + d*cost_per_km

    ic(amt)
    amt += amt*service_tax

    ic(amt)
    amt = round(amt,2)
    print(f"BILL {ride.id} {ride.driver_id} {amt}")


In [20]:
def command_parser(cmd:str):
    cmd, *args = cmd.split(" ")

    if cmd == "ADD_DRIVER":
        coord = list(map(int,args[1:]))
        add_driver(args[0],*coord)

    if cmd == "ADD_RIDER":
        coord = list(map(int,args[1:]))
        add_rider(args[0],*coord)

    if cmd == "MATCH":
        match(args[0])

    if cmd == "START_RIDE":
        ride_id, nth_rider, rider_id = args
        start_ride(ride_id,int(nth_rider),rider_id)

    if cmd == "STOP_RIDE":
        ride_id, dest_x, dest_y, time_taken = args
        dest_coord  = (int(dest_x),int(dest_y))
        stop_ride(ride_id,dest_coord,int(time_taken))

    if cmd == "BILL":
        ride_id = args[0]
        bill(ride_id)

In [47]:
rest_global()
for i in testcase:
    ic(i)
    command_parser(i)


ic| i: 'ADD_DRIVER D1 0 1'
ic| driver: Driver(id='D1', coord=[0, 1])
ic| i: 'ADD_DRIVER D2 2 3'
ic| driver: Driver(id='D2', coord=[2, 3])
ic| i: 'ADD_RIDER R1 3 5'
ic| rider: Rider(id='R1', coord=[3, 5])
ic| i: 'ADD_DRIVER D3 4 2'
ic| driver: Driver(id='D3', coord=[4, 2])
ic| i: 'ADD_RIDER R2 1 1'
ic| rider: Rider(id='R2', coord=[1, 1])
ic| i: 'MATCH R1'
ic| f"ignoring {driver.id} is {d} away": 'ignoring D1 is 5.0 away'
ic| rider_id: 'R1', driver.id: 'D2', d: 2.23606797749979
ic| rider_id: 'R1', driver.id: 'D3', d: 3.1622776601683795
ic| drivers_available: [('D2', 2.23606797749979), ('D3', 3.1622776601683795)]
ic| i: 'MATCH R2'
ic| rider_id: 'R2', driver.id: 'D1', d: 1.0
ic| rider_id: 'R2', driver.id: 'D2', d: 2.23606797749979
ic| rider_id: 'R2', driver.id: 'D3', d: 3.1622776601683795
ic| drivers_available: [('D1', 1.0), ('D2', 2.23606797749979), ('D3', 3.1622776601683795)]
ic| i: 'START_RIDE RIDE-101 1 R1'
ic| i: 'START_RIDE RIDE-102 1 R2'
ic| i: 'STOP_RIDE RIDE-101 10 2 48'
ic| ride:

DRIVERS_MATCHED D2 D3
DRIVERS_MATCHED D1 D2 D3
RIDE_STARTED RIDE-101
RIDE_STARTED RIDE-102


',
               driver_id='D2',
               rider_id='R1',
               start_coords=[3, 5],
               dest_coords=(10, 2),
               started_at=datetime.datetime(2023, 5, 18, 22, 45, 23, 373658),
               ended_at=datetime.datetime(2023, 5, 18, 22, 45, 23, 403450),
               duration=48)
ic| i: 'STOP_RIDE RIDE-102 7 9 50'
ic| ride: Ride(id='RIDE-102',
               driver_id='D1',
               rider_id='R2R2

In [22]:
# ic.disable()
# ic.enable()

In [46]:
def run_testcase(in_str:str):
    rest_global()
    testcase = in_str.strip().splitlines()
    for i in testcase:
        ic(i)
        command_parser(i)

In [34]:
testcase = """
ADD_DRIVER D1 0 1
ADD_DRIVER D2 2 3
ADD_RIDER R1 3 5
ADD_DRIVER D3 4 2
ADD_RIDER R2 1 1
MATCH R1
MATCH R2
START_RIDE RIDE-101 1 R1
START_RIDE RIDE-102 1 R2
STOP_RIDE RIDE-101 10 2 48
STOP_RIDE RIDE-102 7 9 50
BILL RIDE-101
BILL RIDE-102
""".strip().splitlines()

In [33]:
stop

NameError: name 'stop' is not defined

In [25]:
import os

testcase_dir = "sample_input"
testcase_files = os.listdir(testcase_dir)

input_files = [os.path.join(testcase_dir,i) for i in testcase_files if "input" in i ]#and os.path.isfile(i)]
output_files = [os.path.join(testcase_dir,i) for i in testcase_files if "output" in i] #and os.path.isfile(i)]

input_files.sort()
output_files.sort()

In [26]:
input_files,output_files

(['sample_input/input.txt', 'sample_input/input1.txt'],
 ['sample_input/output.txt', 'sample_input/output1.txt'])

In [27]:
testcases = []
for i in range(len(input_files)):
    with open(input_files[i],'r') as f1:
        testcase = [i.strip() for i in f1.readlines() if len(i)>0]
        testcases.append(testcase)



In [28]:
testcases

[['ADD_DRIVER D1 1 1',
  'ADD_DRIVER D2 4 5',
  'ADD_DRIVER D3 2 2',
  'ADD_RIDER R1 0 0',
  'MATCH R1',
  'START_RIDE RIDE-001 2 R1',
  'STOP_RIDE RIDE-001 4 5 32',
  'BILL RIDE-001'],
 ['ADD_DRIVER D1 0 1',
  'ADD_DRIVER D2 2 3',
  'ADD_RIDER R1 3 5',
  'ADD_DRIVER D3 4 2',
  'ADD_RIDER R2 1 1',
  'MATCH R1',
  'MATCH R2',
  'START_RIDE RIDE-101 1 R1',
  'START_RIDE RIDE-102 1 R2',
  'STOP_RIDE RIDE-101 10 2 48',
  'STOP_RIDE RIDE-102 7 9 50',
  'BILL RIDE-101',
  'BILL RIDE-102']]

In [29]:
for i in range(len(testcases)):
    t = "\n".join(testcases[i])
    run_testcase(t)

ic| i: 'ADD_DRIVER D1 1 1'
ic| driver: Driver(id='D1', coord=[1, 1])
ic| i: 'ADD_DRIVER D2 4 5'
ic| driver: Driver(id='D2', coord=[4, 5])
ic| i: 'ADD_DRIVER D3 2 2'
ic| driver: Driver(id='D3', coord=[2, 2])
ic| i: 'ADD_RIDER R1 0 0'
ic| rider: Rider(id='R1', coord=[0, 0])
ic| i: 'MATCH R1'
ic| rider_id: 'R1', driver.id: 'D1', d: 1.4142135623730951
ic| f"ignoring {driver.id} is {d} away": 'ignoring D2 is 6.4031242374328485 away'
ic| rider_id: 'R1', driver.id: 'D3', d: 2.8284271247461903
ic| drivers_available: [('D1', 1.4142135623730951), ('D3', 2.8284271247461903)]
ic| i: 'START_RIDE RIDE-001 2 R1'
ic| i: 'STOP_RIDE RIDE-001 4 5 32'
ic| ride: Ride(id='RIDE-001',
               driver_id='D3',
               rider_id='R1',
               start_coords=[0, 0],
               dest_coords=(4, 5),
               started_at=datetime.datetime(2023, 5, 18, 22, 44, 12, 297831),
               ended_at=datetime.datetime(2023, 5, 18, 22, 44, 12, 314466),
               duration=32)
ic| i: 'BILL RID

DRIVERS_MATCHED D1 D3
RIDE_STARTED RIDE-001
RIDE_STOPPED RIDE-001


amt: 155.6
ic| amt: 186.72
ic| i: 'ADD_DRIVER D1 0 1'
ic| driver: Driver(id='D1', coord=[0, 1])
ic| i: 'ADD_DRIVER D2 2 3'
ic| driver: Driver(id='D2', coord=[2, 3])
ic| i: 'ADD_RIDER R1 3 5'
ic| rider: Rider(id='R1', coord=[3, 5])
ic| i: 'ADD_DRIVER D3 4 2'
ic| driver: Driver(id='D3', coord=[4, 2])
ic| i: 'ADD_RIDER R2 1 1'
ic| rider:

BILL RIDE-001 D3 186.72


 Rider(id='R2', coord=[1, 1])
ic| i: 'MATCH R1'
ic| f"ignoring {driver.id} is {d} away": 'ignoring D1 is 5.0 away'
ic| rider_id: 'R1', driver.id: 'D2', d: 2.23606797749979
ic| rider_id: 'R1', driver.id: 'D3', d: 3.1622776601683795
ic| drivers_available: [('D2', 2.23606797749979), ('D3', 3.1622776601683795)]
ic| i: 'MATCH R2'
ic| rider_id: 'R2', driver.id: 'D1', d: 1.0
ic| rider_id: 'R2', driver.id: 'D2', d: 2.23606797749979


DRIVERS_MATCHED D2 D3


ic| rider_id: 'R2', driver.id: 'D3', d: 3.1622776601683795
ic| drivers_available: [('D1', 1.0), ('D2', 2.23606797749979), ('D3', 3.1622776601683795)]
ic| i: 'START_RIDE RIDE-101 1 R1'
ic| i: 'START_RIDE RIDE-102 1 R2'
ic| i: 'STOP_RIDE RIDE-101 10 2 48'
ic| ride: Ride(id='RIDE-101',
               driver_id='D2',
               rider_id='R1',
               start_coords=[3, 5],
               dest_coords=(10, 2),
               started_at=datetime.datetime(2023, 5, 18, 22, 44, 13, 275190),
               ended_at=datetime.datetime(2023, 5, 18, 22, 44, 13, 305529),
               duration=48)
ic| i: 'STOP_RIDE RIDE-102 7 9 50'


DRIVERS_MATCHED D1 D2 D3
RIDE_STARTED RIDE-101
RIDE_STARTED RIDE-102
RIDE_STOPPED RIDE-101


ic| ride: Ride(id='RIDE-102',
               driver_id='D1',
               rider_id='R2',
               start_coords=[1, 1],
               dest_coords=(7, 9),
               started_at=datetime.datetime(2023, 5, 18, 22, 44, 13, 290867),
               ended_at=datetime.datetime(2023, 5, 18, 22, 44, 13, 460192),
               duration=50)
ic| i: 'BILL RIDE-101'
ic| amt: 195.53
ic| amt: 234.636
ic| i: 'BILL RIDE-102'
ic| amt: 215.0
ic| amt: 258.0


RIDE_STOPPED RIDE-102
BILL RIDE-101 D2 234.64
BILL RIDE-102 D1 258.0


In [30]:
import sys

In [31]:
py_version = sys.version_info

In [32]:
print(py_version.major,py_version.minor)

3 10
